# Сразу оговорюсь
> В этом компетишене я использовала другие ноты, гитхаб, гугл, яндекс и arxiv.org


# Подготовка
> Импорт всего что нужно и не очень. Загрузка данных, необходимые функции

In [ ]:
# вспомогательные
import math, os, re, warnings, random 
# наша основаная библиотека
import tensorflow as tf 
# нампи без него никак
import numpy as np 
# пандас как и нампи 
import pandas as pd 
# это библиотека загрузки и обработки звуков
import librosa
# функции ввода-вывода тензорфлоу
import tensorflow.io 
# библиотека Каггла, она нам нужна чтобы получить пути до данных
from kaggle_datasets import KaggleDatasets  
# рисунки графики и красота
import matplotlib.pyplot as plt 
# ещё рисунки графики и красота для звуков
from IPython.display import Audio 
# мы будем использовать керас
from tensorflow.keras import Model, layers 
# будем разбивать данные на фолды
from sklearn.model_selection import KFold 
#
import tensorflow.keras.backend as K # 
# колбэки для ранней остановки, промежуточных сохранений и контроля lr
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler 
# уровни нейросеть из кераса, которые нам понадобятся, как дополнительные скорее, в основном мы будем пользоваться ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense, Dropout, GaussianNoise
# ResNet50 основная модель, которую мы будем использовать,
# она хороша зарекомендовала себя для распознования изображений
# у неё хорошая эффективность цена/качество
# мы будем использовать её для распознавания звуков
# в качестве функции потерь мы будем использовать кросс-энетропию
# вообще мы могли попробовать создавать и свои модели
# c учётом особенностей данных и наших требований, но времени нет на это
from tensorflow.keras.applications import ResNet50
# во многих решенных нотах на каггле используются efficientnet модели
# но я не настолько хорошо их знаю, чтобы сейчас разбираться с ней ещё
# import efficientnet.keras as efn
import seaborn as sns # это знаменитая библиотека для графиков, тепловых карт и т.д.

In [ ]:
# выводим версию текущей сборки TF
# это скорее перестраховка, в данном случае
print(tf.__version__)

In [ ]:
# получаем данные истинно позитивной тренинговой выборки
# есть ещё и ложно-позитивная, но я её не трогала, т.к. не знаю что с ней делать. 
#Из опыта музыкальных дисктантов и изучения ин. языков, я могу предположить, что она нужна 
#для лучшего распознавания схожих звуков, но не знаю, как это делать
traindf = pd.read_csv(r'../input/rfcx-species-audio-detection/train_tp.csv')
traindf.head()

In [ ]:
# функции получения фич формата tfrecords из значений
# максимально стандартный код
# он вроде бы родом из официального мануала даже

# в этой функции мы получаем фичу с типом BytesList
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# в этой функции мы получаем фичу с типом FloatList
def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

# в этой функции мы сериализуем данные переданные в функцию
# в строчку для будущей записи в файл формата tfrecords
# каждая строка данных там иммет тип tf.train.Example так что делаем через неё
def serialize_example(wav, recording_id, target, song_id, tmin,fmin, tmax, fmax):
    feature = {
      'wav': _bytes_feature(wav),
      'recording_id': _bytes_feature(recording_id),
      'target': _float_feature(target),
      'song_id': _float_feature(song_id),
      'tmin': _float_feature(tmin),
      'fmin' : _float_feature(fmin),
      'tmax': _float_feature(tmax),
      'fmax' : _float_feature(fmax),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    
    return example_proto.SerializeToString() 


In [ ]:
from sklearn.model_selection import StratifiedKFold
# здесь мы должны были создать новые файлики tfrecords
# для тренировки нейросети, но не получилось
# проблема в том, что TPU может читать данные только из input
# а прямого доступа к input нет
# по умолчанию файлики создаются в папке output
# по этому я сделала отдельный ноте, который создал файлики для тренировки
# и подключили результаты его выполнения, как исходные(input) для этого ноте
# и так сработало :)


# tfrec_num = 0
# kfold = StratifiedKFold(n_splits=10, shuffle=False)
# for fold, (train_idx, test_idx) in enumerate(kfold.split(traindf['recording_id'], traindf['species_id'])):
#     x_train , y_train = traindf['recording_id'][test_idx] , traindf['species_id'][test_idx]
   
#     with tf.io.TFRecordWriter('tp%.2i-%.2i.tfrec'%(tfrec_num, len(test_idx))) as writer:
#         print('Writing_tfrecords ',fold)
#         for recording_id , true_value in zip(x_train, y_train): 
#             wav, _ = librosa.load(f'../input/rfcx-species-audio-detection/train/{recording_id}.flac', sr = None)
#             label_info = traindf.loc[traindf['recording_id'] == str(recording_id)].values[0]
#             wav = tf.audio.encode_wav(tf.reshape(wav,(wav.shape[0], 1)) ,sample_rate = 48000)
#             recording_id = label_info[0].encode()
#             target = label_info[1]
#             song_id = label_info[2]
#             tmin = label_info[3]
#             fmin = label_info[4]
#             tmax = label_info[5]
#             fmax = label_info[6]
#             example = serialize_example(wav, recording_id, target, song_id, tmin,fmin, tmax, fmax)
#             writer.write(example)
#     tfrec_num += 1


In [ ]:
# максимально стандартный код для всех блокнотов на Каггле
# мы подключаем TPU или distribute стратегию в зависимости от доступности
# distribute может работать как с GPU так и с CPU 

# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
# здесь мы получаем список файлов с путями для нужных нам данных
# в данном случае frog-song-data - это то что мы расчитали
# в другом блокноте для тренинговых данных
TRAIN_DATA_DIR = 'frog-song-data'
TRAIN_GCS_PATH =  KaggleDatasets().get_gcs_path(TRAIN_DATA_DIR)
FILENAMES = tf.io.gfile.glob(TRAIN_GCS_PATH + '/tp*.tfrec')

In [ ]:
#а это наши тетсовые данные, они загружаются как и предыдущие
TEST_DATA_DIR = 'rfcx-species-audio-detection'
TEST_GCS_PATH =  KaggleDatasets().get_gcs_path(TEST_DATA_DIR)
TEST_FILES = tf.io.gfile.glob(TEST_GCS_PATH + '/tfrecords/test/*.tfrec')

In [ ]:
# последний номер в имени сгенерированного файла означает номер семпла
# мы считаем число семплов через число файлов
# не очень красивый ход... но рабочий
n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in FILENAMES]
print(f"Total number of rows :{np.sum(n)}")
no_of_training_samples=np.sum(n)

In [ ]:
# устанавливаем парметры наших скриптов
CUT = 10
# окно в сек. для всех дорожек
TIME = 10
# число эпох
EPOCHS = 25
# размер батча
GLOBAL_BATCH_SIZE = 4 * REPLICAS
# шаг оптимизатора базовый(он будет меняться)
LEARNING_RATE = 0.0015
# параметр отжига(перегрева) функция колбэк, которая будет менять на lr 
WARMUP_LEARNING_RATE = 1e-5
# параметр отжига(перегрева) функция колбэк, число эпох где lr может расти
WARMUP_EPOCHS = int(EPOCHS*0.1)
PATIENCE = 8
# за сколько шагов загружаем данные в TPU
STEPS_PER_EPOCH = 64
#Количество фолдов, по которым будем обучать
N_FOLDS = 5
NUM_TRAINING_SAMPLES = no_of_training_samples

In [ ]:
# ещё параметры
# это параметры спектрограммы, мы будем вытаскивать дополнительную информацию
# из звука для его распознования и это как бы основная задача и проблема
# именно здесь будут запрятаны основные не однозначности и непонятности
class params:
    # это число байт в секунду для нашей аудио записи
    sample_rate = 48000
    # это размер окна раскладываемое в Фурье в секундах
    stft_window_seconds: float = 0.025
    stft_hop_seconds: float = 0.005
    # длинна окна в байтах = sample_rate * stft_window_seconds
    frame_length: int =  1200    
    # мелы - как хорошо, что я ходила в музыкалку в детстве
    # но даже это не всегда спасает :)
    # мелы - это величина в музыки, она логарифмически связана с частотой
    mel_bands: int = 512
    mel_min_hz: float = 50.0
    mel_max_hz: float = 24000.0
    log_offset: float = 0.001
    patch_window_seconds: float = 0.96
    patch_hop_seconds: float = 0.48

  
    patch_frames =  int(round(patch_window_seconds / stft_hop_seconds))

  
    patch_bands = mel_bands
    height = mel_bands
    width = 2000
    # число классов для классификации
    num_classes: int = 24
    # защита от оверифитинга, но это не самые главный способ в нашем случае
    # добавление уровня dropout для "забывания" старых нейронных связей
    # это хорошо
    # но главное, это слой GaussianNoise
    # он добавляет шум нормального распределения к данным input
    # это позволяет нам НЕ обучать нашу модель на шуме
    # который бы иначе повторялся на каждой эпохе
    dropout = 0.35
    #вообще должен быть softmax, но почему-то sigmoid работает лучше, пока не понимаю
    classifier_activation: str = 'sigmoid'

In [ ]:
# описываем фичи в экзамплах(стрчоках) это нужно, чтобы позже с ними работать
classifier_activation: str = 'sigmoid'
feature_description = {
    'wav': tf.io.FixedLenFeature([], tf.string),
    'recording_id': tf.io.FixedLenFeature([], tf.string ),
    'target' : tf.io.FixedLenFeature([], tf.float32),
    'song_id': tf.io.FixedLenFeature([], tf.float32),
     'tmin' : tf.io.FixedLenFeature([], tf.float32),
     'fmin' : tf.io.FixedLenFeature([], tf.float32),
     'tmax' : tf.io.FixedLenFeature([], tf.float32),
     'fmax' : tf.io.FixedLenFeature([], tf.float32),
}
feature_dtype = {
    'wav': tf.float32,
    'recording_id': tf.string,
    'target': tf.float32,
    'song_id': tf.float32,
    't_min': tf.float32,
    'f_min': tf.float32,
    't_max': tf.float32,
    'f_max':tf.float32,
}

# Обработка и подготовка данных
> Получение дополнительной информации из звука (спектограммы, мелограммы). Сворачивание данных(спектограммы, мелограммы) в прямоугольные тензоры для обработки как картинок

In [ ]:
# функция преобразования сырого звука в спектрограмму
# но наша спекторграмма будет не по Гц, а по мелам
def waveform_to_log_mel_spectrogram(waveform,target_or_rec_id):
    """Compute log mel spectrogram patches of a 1-D waveform."""
    # я решила оставить оригинальные коментарии
    # waveform has shape [<# samples>]

    # Convert waveform into spectrogram using a Short-Time Fourier Transform.
    # Note that tf.signal.stft() uses a periodic Hann window by default.
    
    # Получаем параметры окна для передачи данных в tf.signal.stft
    # для получения спектограммы мощности звука от Гц пока в Герцах
    window_length_samples = int(round(params.sample_rate * params.stft_window_seconds))
    hop_length_samples = int(round(params.sample_rate * params.stft_hop_seconds))
    fft_length = 2 ** int(np.ceil(np.log(window_length_samples) / np.log(2.0)))
    num_spectrogram_bins = fft_length // 2 + 1
    # передаём параметры и получаем спектрограмму (Герцы)
    spectogram=tf.signal.stft(signals=waveform,frame_length=params.frame_length,frame_step=hop_length_samples,fft_length= fft_length)
    magnitude_spectrogram = tf.abs(spectogram)
    # magnitude_spectrogram has shape [<# STFT frames>, num_spectrogram_bins]

    # Convert spectrogram into log mel spectrogram
    # теперь конвертируем спектограмму из Герц в мелы
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=params.mel_bands,
        num_spectrogram_bins=num_spectrogram_bins,
        sample_rate=params.sample_rate,
        lower_edge_hertz=params.mel_min_hz,
        upper_edge_hertz=params.mel_max_hz)
    
    mel_spectrogram = tf.matmul(magnitude_spectrogram, linear_to_mel_weight_matrix)
    
    log_mel = tf.math.log(mel_spectrogram + params.log_offset)
    log_mel = tf.transpose(log_mel)
    log_mel_spectrogram = tf.reshape(log_mel , [tf.shape(log_mel)[0] ,tf.shape(log_mel)[1],1])  
    # ура, теперь у нас есть спектограмма в мелах
    # важно заметить, что все операции по конвертации мы делали при помощи функций TF
    # а это значит, что при компилляции будет сохранен граф взаимосвязи этих функций нативно
    # т.е. все эти операции будут преобразованы в тензорные преобразования TF
    # что очень важно, т.к. операции тензорных преобразований будут неразрывны загружаться в TPU\GPU
    # сейчас для нас это может и не иметь какого-то дикого эффекта, но при работе модели в проде
    # это скажется очень сильно в т.ч.и на неочевидном её поведении
    # надо стараться не выходить за пределы функций TF
    # даже в преобразовании данных =)
    
    # Frame spectrogram (shape [<# STFT frames>, params.mel_bands]) into patches
    # (the input examples). Only complete frames are emitted, so if there is
    # less than params.patch_window_seconds of waveform then nothing is emitted
    # (to avoid this, zero-pad before processing).
    spectrogram_hop_length_samples = int(round(params.sample_rate * params.stft_hop_seconds))
    spectrogram_sample_rate = params.sample_rate / spectrogram_hop_length_samples
    patch_window_length_samples = int(round(spectrogram_sample_rate * params.patch_window_seconds))
    patch_hop_length_samples = int(round(spectrogram_sample_rate * params.patch_hop_seconds))
    features = tf.signal.frame(signal=log_mel_spectrogram,frame_length=patch_window_length_samples,frame_step=patch_hop_length_samples,axis=0)
    # features has shape [<# patches>, <# STFT frames in an patch>, params.mel_bands]
    return log_mel_spectrogram, target_or_rec_id

In [ ]:
# Это все изменение изображений для того, чтобы увеличить нашу выбоку, количество данных
# чтобы модель лучше обучалась, лучше распознавая картинки спектограммы

def frequency_masking(mel_spectrogram):
    
    frequency_masking_para = 80, 
    frequency_mask_num = 2
    
    fbank_size = tf.shape(mel_spectrogram)
#     print(fbank_size)
    n, v = fbank_size[0], fbank_size[1]

    for i in range(frequency_mask_num):
        f = tf.random.uniform([], minval=0, maxval= tf.squeeze(frequency_masking_para), dtype=tf.int32)
        v = tf.cast(v, dtype=tf.int32)
        f0 = tf.random.uniform([], minval=0, maxval= tf.squeeze(v-f), dtype=tf.int32)

        # warped_mel_spectrogram[f0:f0 + f, :] = 0
        mask = tf.concat((tf.ones(shape=(n, v - f0 - f,1)),
                          tf.zeros(shape=(n, f,1)),
                          tf.ones(shape=(n, f0,1)),
                          ),1)
        mel_spectrogram = mel_spectrogram * mask
    return tf.cast(mel_spectrogram, dtype=tf.float32)


def time_masking(mel_spectrogram):
    time_masking_para = 40, 
    time_mask_num = 1
    
    fbank_size = tf.shape(mel_spectrogram)
    n, v = fbank_size[0], fbank_size[1]

   
    for i in range(time_mask_num):
        t = tf.random.uniform([], minval=0, maxval=tf.squeeze(time_masking_para), dtype=tf.int32)
        t0 = tf.random.uniform([], minval=0, maxval= n-t, dtype=tf.int32)

        # mel_spectrogram[:, t0:t0 + t] = 0
        mask = tf.concat((tf.ones(shape=(n-t0-t, v,1)),
                          tf.zeros(shape=(t, v,1)),
                          tf.ones(shape=(t0, v,1)),
                          ), 0)
        
        mel_spectrogram = mel_spectrogram * mask
    return tf.cast(mel_spectrogram, dtype=tf.float32)



def random_brightness(image):
    return tf.image.random_brightness(image, 0.2)

def random_gamma(image):
    return tf.image.random_contrast(image, lower = 0.1, upper = 0.3)

def random_flip_right(image):
    return tf.image.random_flip_left_right(image)

def random_flip_up_down(image):
    return tf.image.random_flip_left_right(image)

available_ops = [
          frequency_masking ,
          time_masking, 
          random_brightness, 
          random_flip_up_down,
          random_flip_right 
         ]

#выбираем от нуля до трех преобразований
def apply_augmentation(image, target):
    num_layers = int(np.random.uniform(low = 0, high = 3))
    
    for layer_num in range(num_layers):
        op_to_select = tf.random.uniform([], maxval=len(available_ops), dtype=tf.int32, seed = seed)
        for (i, op_name) in enumerate(available_ops):
            image = tf.cond(
            tf.equal(i, op_to_select),
            lambda selected_func=op_name,: selected_func(
                image),
            lambda: image)
    return image, target

In [ ]:
# преобразуем в картинку для распознавания
def preprocess(image, target_or_rec_id):
    
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, [params.height,params.width])
    image = tf.image.per_image_standardization(image)
    return image , target_or_rec_id

# мы обрезаем наши wav (кодированные записи) по нашим доп параметра tmin max, fmin max и возвращаем 
# и возвращаем только данные, содержащие закодированный wav  и target
def read_labeled_tfrecord(example_proto):
    # Обратно разделяем строчки на столбцы
    sample = tf.io.parse_single_example(example_proto, feature_description)
    wav, _ = tf.audio.decode_wav(sample['wav'], desired_channels=1) # mono
    # каст гарантирует, что будет тип float
    target = tf.cast(sample['target'],tf.float32)
    #делаем матрицу классов, как процедура дамми переменных, только с тензорными операциями
    target = tf.squeeze(tf.one_hot([target,], depth = params.num_classes), axis = 0)
    
    tmin = tf.cast(sample['tmin'], tf.float32)
    fmin = tf.cast(sample['fmin'], tf.float32)
    tmax = tf.cast(sample['tmax'], tf.float32)
    fmax = tf.cast(sample['fmax'], tf.float32)
    # Макс и мин получаем в байтах
    tmax_s = tmax * tf.cast(params.sample_rate, tf.float32)
    tmin_s = tmin * tf.cast(params.sample_rate, tf.float32)
    # считаем байты
    cut_s = tf.cast(CUT * params.sample_rate, tf.float32)
    all_s = tf.cast(60 * params.sample_rate, tf.float32)
    tsize_s = tmax_s - tmin_s
    #сколько надо отрезать слева
    cut_min = tf.cast(
    tf.maximum(0.0, 
        tf.minimum(tmin_s - (cut_s - tsize_s) / 2,
                   tf.minimum(tmax_s + (cut_s - tsize_s) / 2, all_s) - cut_s)
    ), tf.int32
      )
    #сколько надо отрезать справа
    cut_max = cut_min + CUT * params.sample_rate
    wav = tf.squeeze(wav[cut_min : cut_max] )
    
    return wav, target

# Это функция для загрузки тестовых данных
def read_unlabeled_tfrecord(example):
    feature_description = {
    'recording_id': tf.io.FixedLenFeature([], tf.string),
    'audio_wav': tf.io.FixedLenFeature([], tf.string),
    }
    sample = tf.io.parse_single_example(example, feature_description)
    wav, _ = tf.audio.decode_wav(sample['audio_wav'], desired_channels=1) # mono
    recording_id = tf.reshape(tf.cast(sample['recording_id'] , tf.string), [1])
#     wav = tf.squeeze(wav)

# Обрезаем тестовое аудио
    def _cut_audio(i):
        _sample = {
            'audio_wav': tf.reshape(wav[i*params.sample_rate*TIME:(i+1)*params.sample_rate*TIME], [params.sample_rate*TIME]),
            'recording_id': sample['recording_id']
        }
        return _sample

    return tf.map_fn(_cut_audio, tf.range(60//TIME), dtype={
        'audio_wav': tf.float32,
        'recording_id': tf.string
    })

In [ ]:
# это функция загрузки данных по массиву файлов из tfrecords
def load_dataset(filenames, labeled = True, ordered = False , training = True):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    ignore_order = tf.data.Options()
    if not ordered:
        # disable order, increase speed
        ignore_order.experimental_deterministic = False
        
#     GCS_PATH = KaggleDatasets().get_gcs_path('audio-note-generated')
#     print("GCS_PATH: "+GCS_PATH)
    # automatically interleaves reads from multiple files
    
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO )
    # use data as soon as it streams in, rather than in its original order
    # Возвращаем черем map ко всем значениям датасета функцию и возвращаем результаты выполнения
    # Возвращвется wav и target
    dataset = dataset.map(read_labeled_tfrecord , num_parallel_calls = AUTO )
    #превращаем в мел спектограмму
    dataset = dataset.map(waveform_to_log_mel_spectrogram , num_parallel_calls = AUTO)   
    if training:
        # применяем орментацию (портим данные) для лувчшего качества
        dataset = dataset.map(apply_augmentation, num_parallel_calls = AUTO)
        # делаем картинку
    dataset = dataset.map(preprocess, num_parallel_calls = AUTO)
    return dataset


In [ ]:
# обертка для функции выше с разбитием на батчи, чтобы загрузить в TPU
def get_dataset(filenames, training = True):
    if training:
        dataset = load_dataset(filenames , training = True)
        dataset = dataset.shuffle(256).repeat()
        dataset = dataset.batch(GLOBAL_BATCH_SIZE, drop_remainder = True)
    else:
        dataset = load_dataset(filenames , training = False)
        dataset = dataset.batch(GLOBAL_BATCH_SIZE).cache()
    
    dataset = dataset.prefetch(AUTO)
    return dataset


In [ ]:
# стандартная функция установки ядер случайных величин, чтобы случайные величины всегды были одинаковыми
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 123
seed_everything(seed)
warnings.filterwarnings('ignore')


In [ ]:
# это я чисто тестила, у меня эта функция заработала не сразу, мягко говоря
train_dataset = get_dataset(FILENAMES, training = True)


# Определение метрики и колбэка для всего

In [ ]:
# здесь мы создаём метрику для нашей модели(не ошибку), просто метрику
# она сделана по описанию компетишина, такой она применяется при выборе места участника
# чем эта метрика выше, тем лучше место, максимум = 1 и минимум = 0
# однако её алгоритм не обычен для меня, я не уверена, что она прямо удовлетворяет
# условиям метрики 
def _one_sample_positive_class_precisions(example):
    y_true, y_pred = example
    y_true = tf.reshape(y_true, tf.shape(y_pred))
    retrieved_classes = tf.argsort(y_pred, direction='DESCENDING')
#     shape = tf.shape(retrieved_classes)
    class_rankings = tf.argsort(retrieved_classes)
    retrieved_class_true = tf.gather(y_true, retrieved_classes)
    retrieved_cumulative_hits = tf.math.cumsum(tf.cast(retrieved_class_true, tf.float32))

    idx = tf.where(y_true)[:, 0]
    i = tf.boolean_mask(class_rankings, y_true)
    r = tf.gather(retrieved_cumulative_hits, i)
    c = 1 + tf.cast(i, tf.float32)
    precisions = r / c

    dense = tf.scatter_nd(idx[:, None], precisions, [y_pred.shape[0]])
    return dense

# @tf.function
class LWLRAP(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='lwlrap'):
        super().__init__(name=name)

        self._precisions = self.add_weight(
            name='per_class_cumulative_precision',
            shape=[num_classes],
            initializer='zeros',
        )

        self._counts = self.add_weight(
            name='per_class_cumulative_count',
            shape=[num_classes],
            initializer='zeros',
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        precisions = tf.map_fn(
            fn=_one_sample_positive_class_precisions,
            elems=(y_true, y_pred),
            dtype=(tf.float32),
        )

        increments = tf.cast(precisions > 0, tf.float32)
        total_increments = tf.reduce_sum(increments, axis=0)
        total_precisions = tf.reduce_sum(precisions, axis=0)

        self._precisions.assign_add(total_precisions)
        self._counts.assign_add(total_increments)        

    def result(self):
        per_class_lwlrap = self._precisions / tf.maximum(self._counts, 1.0)
        per_class_weight = self._counts / tf.reduce_sum(self._counts)
        overall_lwlrap = tf.reduce_sum(per_class_lwlrap * per_class_weight)
        return overall_lwlrap

    def reset_states(self):
        self._precisions.assign(self._precisions * 0)
        self._counts.assign(self._counts * 0)


https://github.com/Tony607/Keras_Bag_of_Tricks/blob/master/warmup_cosine_decay_scheduler.py

https://arxiv.org/abs/1608.03983

In [ ]:
# это хак из Keras_Bag_of_Tricks помогает контролировать lr
# вроде бы это должно приводить к лучшей сходимости
# в начале мы позволяем lr росте до warmup_steps
# а после lr падает причём в соответствии с куском синусоиды
# ниже будет график с примером
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps= 0,
                             hold_base_rate_steps=0):
 
    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                     'warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + tf.cos(
        np.pi *
        (tf.cast(global_step, tf.float32) - warmup_steps - hold_base_rate_steps
        ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = tf.where(
          global_step > warmup_steps + hold_base_rate_steps,
          learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                         'warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * tf.cast(global_step,
                                    tf.float32) + warmup_learning_rate
        learning_rate = tf.where(global_step < warmup_steps, warmup_rate,
                               learning_rate)
    return tf.where(global_step > total_steps, 0.0, learning_rate,
                    name='learning_rate')


#dummy example
rng = [i for i in range(int(EPOCHS * STEPS_PER_EPOCH))]
WARMUP_STEPS =  int(WARMUP_EPOCHS * STEPS_PER_EPOCH)
y = [cosine_decay_with_warmup(x , LEARNING_RATE, len(rng), 1e-5, WARMUP_STEPS) for x in rng]

sns.set(style='whitegrid')
fig, ax = plt.subplots(figsize=(20, 6))
plt.plot(rng, y)


In [ ]:
# to apply learning rate schedule stepwise we need to subclass keras callback
# if we would have applied lr schedule epoch wise then it is not needed we can only call class learningrateschedule 
# это callback обертка над функцией выше, она нужна, чтобы керас съел это как callback
class WarmUpCosineDecayScheduler(tf.keras.callbacks.Callback):

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr.numpy()))
            

total_steps = int(EPOCHS * STEPS_PER_EPOCH)
# Compute the number of warmup batches or steps.
warmup_steps = int(WARMUP_EPOCHS * STEPS_PER_EPOCH)
warmup_learning_rate = WARMUP_LEARNING_RATE


In [ ]:
#А это собственно наша модель 
def ResNet50_MODEL():
    waveform = Input(shape=(None,None,3), dtype=tf.float32)
    noisy_waveform = GaussianNoise(0.2)(waveform)
    model = ResNet50(include_top=False, weights='imagenet',) 
    model_output = model(noisy_waveform)
    model_output = GlobalAveragePooling2D()(model_output)
    dense = Dropout(params.dropout)(model_output)
    predictions = Dense(params.num_classes, activation = params.classifier_activation )(dense)
    model = Model(
      name='ResNet50', inputs=waveform,
      outputs=[predictions])
    return model


In [ ]:
# создаем модель и компилируем с оптимизатором
# бинарной кросэнтропией как функцией потерь
# бинарная кросэтропия выбрана потому что у нас простая класификация
# для каждого из классов есть только два возможных варианта
# либо звук принадлежит этому классу, либо нет
# и метрикой компетишена LWLRAP
def get_model():
    with strategy.scope():
        model = ResNet50_MODEL()
        model.summary()
        model.compile(optimizer = 'adam',
                                loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.1),
                                metrics = [LWLRAP(num_classes = params.num_classes),
                                ])
    return model

In [ ]:

skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=seed)
oof_pred = []; oof_labels = []; history_list = []

for fold,(idxT, idxV) in enumerate(skf.split(np.arange(10))):
    if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    print(f'\nFOLD: {fold+1}')
    print(f'TRAIN: {idxT} VALID: {idxV}')

    # Create train and validation sets
    TRAIN_FILENAMES = [FILENAMES[x] for x in idxT]
    VALID_FILENAMES = [FILENAMES[x] for x in idxV]
    np.random.shuffle(TRAIN_FILENAMES)
    
    train_dataset =  get_dataset(TRAIN_FILENAMES, training=True,)
    validation_data= get_dataset(VALID_FILENAMES, training=False) 
    model = get_model()

    model_path = f'RFCX_model_fold {fold}.h5'
    early_stopping = EarlyStopping(monitor = 'val_lwlrap', mode = 'max', 
                       patience = PATIENCE, restore_best_weights=True, verbose=1)

    # Create the Learning rate scheduler.
    cosine_warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base= LEARNING_RATE,
                                    total_steps= total_steps,
                                    warmup_learning_rate= warmup_learning_rate,
                                    warmup_steps= warmup_steps,
                                    hold_base_rate_steps=0)

    ## TRAIN
    history = model.fit(train_dataset, 
                        steps_per_epoch=STEPS_PER_EPOCH, 
                        callbacks=[early_stopping, cosine_warm_up_lr], 
                        epochs=EPOCHS,  
                        validation_data = validation_data,
                        verbose = 2).history

    history_list.append(history)
    # Save last model weights
    model.save_weights(model_path)

# OOF predictions
#     ds_valid = get_dataset(VALID_FILENAMES, training = False)
#     oof_labels.append([target.numpy() for frame, target in iter(ds_valid.unbatch())])
#     x_oof = ds_valid.map(lambda frames, target: frames)
#     oof_pred.append(np.argmax(model.predict(x_oof), axis=-1))

    ## RESULTS
#     print(f"#### FOLD {fold+1} OOF Accuracy = {np.max(history['val_lwlrap']):.3f}")


In [ ]:
# def plot_history(history):
#     plt.figure(figsize=(8,3))
#     plt.subplot(1,2,1)
#     plt.plot(history["loss"])
#     plt.plot(history["val_loss"])
#     plt.legend(['Train', 'Test'], loc='upper left')
#     plt.title("loss")

#     plt.subplot(1,2,2)
#     plt.plot(history["lwlrap"])
#     plt.plot(history["val_lwlrap"])
#     plt.legend(['Train', 'Test'], loc='upper left')
#     plt.title("lwlrap")
    
# for hist in history_list:
#     plot_history(hist)


In [ ]:
def get_test_dataset(filenames, training = False):
    
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO )  
    dataset = dataset.map(read_unlabeled_tfrecord , num_parallel_calls = AUTO ).unbatch()
    dataset = dataset.map(lambda spec : waveform_to_log_mel_spectrogram(spec['audio_wav'], spec['recording_id']) , num_parallel_calls = AUTO)
    dataset = dataset.map(preprocess, num_parallel_calls = AUTO)
    return dataset.batch(GLOBAL_BATCH_SIZE*4).cache()


In [ ]:
test_predict = []

test_data = get_test_dataset(TEST_FILES, training = False)
test_audio = test_data.map(lambda frames, recording_id: frames)

for fold in range(N_FOLDS):
    model.load_weights(f'./RFCX_model_fold {fold}.h5')
    test_predict.append(model.predict(test_audio, verbose = 1 ))


In [ ]:
# Загружаем sample submission, чтобы потом заменить на наши предсказанные значения, чтобы были в нужном формате для сабмишена
SUB = pd.read_csv('../input/rfcx-species-audio-detection/sample_submission.csv')

predict = np.array(test_predict).reshape(N_FOLDS, len(SUB), 60 // TIME, params.num_classes)
predict = np.mean(np.max(predict ,axis = 2) , axis = 0)
# predict = np.mean(predict, axis =  0)

recording_id = test_data.map(lambda frames, recording_id: recording_id).unbatch()
# # all in one batch
test_ids = next(iter(recording_id.batch(len(SUB) * 60 // TIME))).numpy().astype('U').reshape(len(SUB), 60 // TIME)

pred_df = pd.DataFrame({ 'recording_id' : test_ids[:, 0],
             **{f's{i}' : predict[:, i] for i in range(params.num_classes)} })


In [ ]:
pred_df.sort_values('recording_id', inplace = True) 
pred_df.to_csv('submission.csv', index = False)  